In [1]:
# run()

In [2]:
from agent.base import Agent
from optimization.Optimizer import PyTorchObjective

import matplotlib.pyplot as plt

# gym_gvgai.dir

%matplotlib inline
import gym
import gvgai
from IPython import display

In [3]:
def show_state(env, step=0, name="", info=""):
    """Fn to visualize the agent playing the game in a notebook
    """
    plt.figure(10)
    plt.clf()
    plt.imshow(env.render(mode="rgb_array")[0])
    plt.title("{} | Step: {} {}".format(name, step, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [4]:
from generator.env_gen_wrapper import GridGame

In [5]:
from agent.base import simulate

In [6]:
import numpy as np

In [7]:
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

In [8]:
from agent.NNagent import NNagent

In [9]:
import torch

In [10]:
popLim = 2
population = [None]*popLim

for i in range(len(population)):
    population[i] = NNagent(GridGame(game='zelda', 
                                     play_length=1000,
                                     path='./levels',
                                     lvl_name='start.txt', 
                                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                                     images=False
                                  )
                         )

In [11]:
population.append(NNagent(GridGame(game='zelda', 
                                     play_length=1000,
                                     path='./levels',
                                     lvl_name='start.txt', 
                                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                                     images=False
                                  ), 
                          parent=torch.load("./25_gen_weights_5_5.pt")
                         )
                 )

In [12]:
tile = population[0].reset()

In [37]:
import time
start = time.time()
population[2].evaluate()
end = time.time() - start
print(end)

solved env with sc: 2.0
0.10809850692749023


In [14]:
# population[0].fitness(noisy=True, fn=show_state)

In [15]:
# population[0].env._fit

In [16]:
# def make():
#     def _make():
#         return GridGame(game='zelda', 
#                          play_length=250, 
#                          path=gym_gvgai.dir + '/envs/games/zelda_v0/', 
#                          lvl_name='zelda_lvl0.txt', 
#                          mechanics=['1', '2', '3', '+', 'g', 'w'], # monster, key, door, wall
#                                )
#     return _make

In [17]:
# x = make()()

In [18]:
# x.env.env.get_action_meanings()

In [19]:
# def move(action):
#     a, b, c, d = x.step(action)
#     print(b)
#     plt.imshow(d['pic'])

In [20]:
#  move(2)

In [21]:
# env = [make(), make()]

In [22]:
# from baselines.common.vec_env.shmem_vec_env import ShmemVecEnv

In [23]:
# envs = ShmemVecEnv(env, context='fork')

In [24]:
# envs.step_async([0, 0])

In [25]:
# [pipe.poll() for pipe in envs.parent_pipes]

In [26]:
# if np.all([pipe.poll() for pipe in envs.parent_pipes]):
#     outs = [pipe.recv() for pipe in envs.parent_pipes]

In [27]:
# self.waiting_step = False
# obs, rews, dones, infos = zip(*outs)
# z = (envs._decode_obses(obs), np.array(rews), np.array(dones), infos)

In [28]:
# z = envs2.step_wait()



- kill oldest.   

- Write mutate wrapper that lives at top of Pair that will create a new Pair obj  
- Copy the Agent, mutate the env.  

- Flip fiber bits in the Tensor NN input.  

----  
- Interestingly, POET has a default of `max_children=8` for each reporduce step. 

In [29]:
population

In [30]:
from optimization.Optimizer import run_opt_n_steps

In [31]:
steps_between_transfer = 2
steps_between_mutation = 10

In [38]:
for epochs in range(3):
    for i in range(len(population)):
        population[i].fitness(noisy=True)
    
    print('evaluated')
    if epochs % steps_between_mutation == 0:
        for i in range(len(population)):
            for _ in range(2):
                population.append(population[i].mutate(0.7))
                
        population = sorted(population, key=lambda x: x.env.id, reverse=True)[:10] 
        print(f"parents + children = {len(population)} old + new members")
        
    # sort by id, kill Pairs with smallest IDs. 
    print(f"generation has {len(population)} members")
    
    opt_wrapper = [PyTorchObjective(agent) for agent in population]
    
    print('optimizing')
    for each_pair in opt_wrapper:
        ans = run_opt_n_steps(each_pair, n=2, popsize=24)
        each_pair.update_nn(ans)

        
    if epochs % steps_between_transfer == 0:
        print('transferring')
        # for each environment, i, (and therefore paired agent)
        for i, current_pair in enumerate(population):
            # save paired agent's score and network
            best = (current_pair.nn, current_pair.max_achieved_score)
            # for every other network, evaluate environment i in agent j
            for j, other_pair in enumerate(population):
                if i == j:
                    continue
                else:
                    j_score = current_pair.env.fitness(other_pair)
                    if best[1] >= j_score:
                        # updated network
                        print(f"update network {i} to {j}")
                        best = (other_pair.nn, j_score)
                        
            #transfer into environment, i, the agent, j, who performed the best.
            #
            # Potential new evaluation metric here. We keep track on how many steps the agent takes
            # therefore, what if the final score is equal to earned score / num_steps. 
            # 
            # This would differential agents who achieve the score faster with higher fitness values
            
            current_pair.nn = best[0]
            

evaluated
parents + children = 9 old + new members
generation has 9 members
optimizing
25
25
25
25
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: 0.0
solved env with sc: 0.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: 0.0
solved env with sc: -1.0
solved env with sc: 0.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solved env with sc: -1.0
solve

In [39]:
population

In [57]:
population[8].fitness()

0.0

In [50]:
print(str(population[3].env.generator))

wwwwwwwwwwwww
w...........w
w...........w
w.+...A.....w
w...........w
w...........w
w...1.......w
w.g.........w
wwwwwwwwwwwww



In [58]:
population[0].env.fitness(population[2])

1.0